In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Data Preprocession
- 先增加一個 column **Up** 來標示出隔天收盤價是否為漲，1 代表漲; 0 代表跌

In [4]:
df = pd.read_csv('stock_FDA.csv')
df.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume
0,02-Jan-2009,902.99,931.80,934.73,899.35,4048270080
1,05-Jan-2009,929.17,927.45,936.63,919.53,5413910016
2,06-Jan-2009,931.17,934.70,943.85,927.28,5392620032
3,07-Jan-2009,927.45,906.65,927.45,902.37,4704940032
4,08-Jan-2009,905.73,909.73,910.00,896.81,4991549952


In [5]:
def UpDown(closePrice):
    up = []
    for i in range(len(closePrice[:-1])):
        if closePrice[i] < closePrice[i+1]:
            up.append(1)
        else:
            up.append(0)
    return up

a = UpDown(df['Close Price'])
df = df[:-1]
df['Up'] = a
df.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume,Up
0,02-Jan-2009,902.99,931.80,934.73,899.35,4048270080,0
1,05-Jan-2009,929.17,927.45,936.63,919.53,5413910016,1
2,06-Jan-2009,931.17,934.70,943.85,927.28,5392620032,0
3,07-Jan-2009,927.45,906.65,927.45,902.37,4704940032,1
4,08-Jan-2009,905.73,909.73,910.00,896.81,4991549952,0


# Logistic Regression
- 增加一個 column 來表示明天是漲或是跌

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [52]:
df_ = df.drop('Date',axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_.drop(['Up'],axis=1), 
                                                    df_['Up'], test_size=0.30)
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [39]:
from sklearn.metrics import classification_report
from sklearn import metrics

## Evaluate
- 用最基本的方法直接做發現 Logistic regression 的 model always predict 1

In [201]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       307
           1       0.55      1.00      0.71       372

   micro avg       0.55      0.55      0.55       679
   macro avg       0.27      0.50      0.35       679
weighted avg       0.30      0.55      0.39       679



In [53]:
metrics.accuracy_score(y_test,predictions)

0.5552282768777614

- 嘗試看看有沒有方法讓 model 開始 predict 0
- 發現把 training data 中的 column **Volumn** drop 掉後開始會 predict 0 但次數仍然很少

In [207]:
X_train, X_test, y_train, y_test = train_test_split(df_.drop(['Up','Volume'],axis=1), 
                                                    df_['Up'], test_size=0.30)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))
print("Total number of 0:",len(predictions[predictions == 0]))

              precision    recall  f1-score   support

           0       0.43      0.13      0.20       305
           1       0.55      0.86      0.67       374

   micro avg       0.53      0.53      0.53       679
   macro avg       0.49      0.50      0.44       679
weighted avg       0.50      0.53      0.46       679

Total number of 0: 92


# SVM

In [44]:
X_train, X_test, y_train, y_test = train_test_split(df_.drop(['Up'],axis=1), 
                                                    df_['Up'], test_size=0.30)

In [45]:
from sklearn.preprocessing import StandardScaler

In [46]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [47]:
from sklearn.svm import SVC

In [48]:
svm = SVC()
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [50]:
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       318
           1       0.53      1.00      0.69       361

   micro avg       0.53      0.53      0.53       679
   macro avg       0.27      0.50      0.35       679
weighted avg       0.28      0.53      0.37       679



In [51]:
metrics.accuracy_score(y_test,predictions)

0.5316642120765832

# Neural Network

In [32]:
from keras import models, layers, optimizers, regularizers
from keras.optimizers import RMSprop

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_.drop(['Up'],axis=1), 
                                                    df_['Up'], test_size=0.30)

In [18]:
X_train.head()

,Open Price,Close Price,High Price,Low Price,Volume
1460,1909.38,1941.28,1942.45,1909.38,2504900352
2150,2475.56,2473.45,2477.62,2468.43,1956491520
840,1402.32,1391.57,1403.07,1388.71,2640128512
1182,1685.04,1687.99,1688.73,1682.22,1588863872
78,862.82,857.51,868.83,854.65,5613459968


In [16]:
hidden_units = 10    # how many neurons in the hidden layer
activation = 'relu'  # activation function for hidden layer
l2 = 0.001           # regularization - how much we penalize large parameter values
learning_rate = 0.1  # how big our steps are in gradient descent
epochs = 20          # how many epochs to train for
batch_size = 32      # how many samples to use for each gradient descent update

In [35]:
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# create a sequential model
model = models.Sequential()

# add the hidden layer
model.add(layers.Dense(input_shape=(5,),
                       units=32, 
                       activation=activation))

# add the output layer
model.add(layers.Dense(input_dim=hidden_units,
                       units=1,
                       activation='softmax'))

# define our loss function and optimizer
model.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                192       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [38]:
result = model.fit(X_train, y_train,
                      batch_size=batch_size,
                      epochs=1)

Epoch 1/1
1584/1584 [==============================] - 0s 41us/step - loss: 7.2365 - acc: 0.5461


In [41]:
pred = model.predict(X_test)
metrics.accuracy_score(y_test,pred)

0.5463917525773195

# Compare each method
## 1. Using the baseline model
### Accuracy
- Logistic regression:0.5552282768777614
- SVM:0.5316642120765832
- Neural Network:0.5463917525773195

### Problem 
所有的 model 都只會預測 1，所以 Accuracy 完全沒有參考價值，只是剛好比較多結果是 1 的被分在 testing data

## 2. Try something to optimize